In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from datetime import datetime, timedelta

# 기본 데이터 세팅

In [2]:
블록원데이터 = pd.DataFrame({"블록명":["S1", "S2", "S3", "S4"], 
                    "중량": [50, 60, 30, 20], 
                    "가로": [10, 10, 5, 5], 
                    "세로":[10, 12, 6, 6], 
                    "표준공기":[2, 3, 1, 2], 
                    "납기":["2024-02-15", "2024-02-15", "2024-02-15", "2024-02-15"],
                    "정반배치":[0, 0, 0, 0]})
블록원데이터.head()

,블록명,중량,가로,세로,표준공기,납기,정반배치
0,S1,50,10,10,2,2024-02-15,0
1,S2,60,10,12,3,2024-02-15,0
2,S3,30,5,6,1,2024-02-15,0
3,S4,20,5,6,2,2024-02-15,0


In [3]:
정반원데이터 = pd.DataFrame({"정반명":["A", "B", "C"],
                   "가능중량":[100, 100, 30],
                   "가로":[10, 10, 5],
                   "세로":[20, 10, 10],
                   "occupied":[0, 0, 0]})
정반원데이터.head()

,정반명,가능중량,가로,세로,occupied
0,A,100,10,20,0
1,B,100,10,10,0
2,C,30,5,10,0


In [4]:
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 28)
날짜집합  = pd.date_range(start=start_date, end=end_date, freq='D')
날짜집합

DatetimeIndex(['2024-02-01', '2024-02-02', '2024-02-03', '2024-02-04',
               '2024-02-05', '2024-02-06', '2024-02-07', '2024-02-08',
               '2024-02-09', '2024-02-10', '2024-02-11', '2024-02-12',
               '2024-02-13', '2024-02-14', '2024-02-15', '2024-02-16',
               '2024-02-17', '2024-02-18', '2024-02-19', '2024-02-20',
               '2024-02-21', '2024-02-22', '2024-02-23', '2024-02-24',
               '2024-02-25', '2024-02-26', '2024-02-27', '2024-02-28'],
              dtype='datetime64[ns]', freq='D')

In [5]:
블록집합 = 블록원데이터["블록명"].unique().tolist()
블록집합

['S1', 'S2', 'S3', 'S4']

In [6]:
최초정반집합 = 정반원데이터["정반명"].unique().tolist()
최초정반집합

['A', 'B', 'C']

In [7]:
쌩전체경우의수 = len(날짜집합) * len(블록집합) * len(최초정반집합)
쌩전체경우의수

336

# 블록 데이터 전처리 함수

In [8]:
# df apply 함수에 복수개의 변수를 던져주기
import functools
def unpack_df_columns(func):
    @functools.wraps(func)
    def _unpack_df_columns(*args, **kwargs):
        series = args[0]
        return func(*series.values)
    return _unpack_df_columns

In [9]:
@unpack_df_columns
def 최소요구착수일구하기(납기, 공기):
    result = pd.to_datetime(납기) - timedelta(days=int(공기))
    return result.date()

In [10]:
착수일가중치, 공기가중치, 크기가중치 = 0.7, 0.5, 0.5

@unpack_df_columns
def 블록우선순위구하기(날순, 공순, 크순):
    global 착수일가중치, 공기가중치, 크기가중치
    result = np.round((날순*착수일가중치 + 공순*공기가중치 + 크순*크기가중치)/3,1)
    return result

In [11]:
def 블록데이터전처리(블록원데이터):
    df1 = 블록원데이터.copy()
    df1["납기"] = pd.to_datetime(df1["납기"])
    df1["사이즈"] = df1.eval("가로*세로")
    df1["최소요구착수일"] = df1[["납기", "표준공기"]].apply(최소요구착수일구하기, axis=1)
    df1["날짜순서"] = df1["최소요구착수일"].rank()
    df1["공기순서"] = df1["표준공기"].rank(ascending=False)
    df1["크기순서"] = df1["사이즈"].rank(ascending=False)
    df1["우선순위"] = df1[["날짜순서", "공기순서", "크기순서"]].apply(블록우선순위구하기, axis=1)
    df1 = df1.sort_values(by=["우선순위"])
    return df1

In [12]:
df1 = 블록데이터전처리(블록원데이터)
df1

,블록명,중량,가로,세로,표준공기,납기,정반배치,사이즈,최소요구착수일,날짜순서,공기순서,크기순서,우선순위
1,S2,60,10,12,3,2024-02-15,0,120,2024-02-12,1.0,1.0,1.0,0.6
0,S1,50,10,10,2,2024-02-15,0,100,2024-02-13,2.5,2.5,2.0,1.3
3,S4,20,5,6,2,2024-02-15,0,30,2024-02-13,2.5,2.5,3.5,1.6
2,S3,30,5,6,1,2024-02-15,0,30,2024-02-14,4.0,4.0,3.5,2.2


# 정반 데이터 전처리 함수

In [13]:
중량가중치, 크기가중치 = 0.5, 0.7

@unpack_df_columns
def 정반우선순위구하기(중순, 크순):
    global 중량가중치, 크기가중치
    result = np.round((중순*중량가중치 + 크순*크기가중치)/3,1)
    return result

In [14]:
def 정반데이터전처리(정반원데이터):
    df2 = 정반원데이터.copy()
    df2["사이즈"] = df2.eval("가로 * 세로")
    df2["중량순서"] = df2["가능중량"].rank(ascending=False)
    df2["크기순서"] = df2["사이즈"].rank(ascending=False)
    df2["우선순위"] = df2[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
    df2 = df2.sort_values(by=["우선순위"])
    return df2

In [15]:
df2 = 정반데이터전처리(정반원데이터)
df2

,정반명,가능중량,가로,세로,occupied,사이즈,중량순서,크기순서,우선순위
0,A,100,10,20,0,200,1.5,1.0,0.5
1,B,100,10,10,0,100,1.5,2.0,0.7
2,C,30,5,10,0,50,3.0,3.0,1.2


# 배치달력 함수

In [16]:
def create_init_calendar(날짜집합, 정반집합):
    배치달력 = pd.DataFrame()
    배치달력.index = 날짜집합
   
    for 정반 in 정반집합:
        배치달력[f"{정반}"] = 0

    return 배치달력

In [17]:
def update_배치달력(배치달력, 정반명, 착수날짜, 필요공기, 정반집합):
    
    신규칼럼리스트 = 정반집합.copy()
    try:
        for 현칼럼 in 배치달력.columns:
            신규칼럼리스트.remove(현칼럼)

        for 신규칼럼 in 신규칼럼리스트:
            배치달력[f"{신규칼럼}"] = 0

        시점인덱스 = list(배치달력.index.strftime('%Y-%m-%d')).index(착수날짜)
        배치달력[f"{정반명}"].iloc[시점인덱스:시점인덱스+필요공기] = 1
        return 배치달력
    except:
        return 배치달력

# 공기달력 함수

In [18]:
def create_공기달력(배치달력, 날짜집합, 정반집합):
    total_list = []

    for 정반 in 정반집합:
        검토대상 = 배치달력[f"{정반}"].tolist()

        new_list = []
        new_num = 0
        for idx, i in enumerate(검토대상):
            if i == 0:
                new_num = new_num  + 1
                new_list.append(new_num)
            else:
                new_list.append(0)
                new_num = 0
        total_list.append(new_list)
        
    new_total = []
    for original_list in total_list:

        result_list = []
        group = []
        for num in original_list:
            if num == 0 and group:
                result_list.extend(reversed(group))
                group = []
            group.append(num)

        result_list.extend(reversed(group))

        new_total.append(result_list)

    공기달력 = pd.DataFrame()
    공기달력.index = 날짜집합

    for idx, 정반 in enumerate(정반집합):
        공기달력[f"{정반}"] =  new_total[idx]

    for 정반 in 정반집합:
        if 공기달력[f"{정반}"][0]== 0:
            공기달력[f"{정반}"] = 공기달력[f"{정반}"].shift(1)
        else:
            pass
    공기달력.fillna(0, inplace=True)
    return 공기달력

# 공백순서달력(first_zero) 함수

In [19]:
def create_공백순서달력(배치_달력, 정반집합, 날짜집합):
    total = []

    for 정반 in 배치_달력.columns.tolist():
        
        input_list = 배치_달력[f"{정반}"].tolist()
        
        counter = 1
        result_list = []

        for idx, x in enumerate(input_list):

            if idx == 0:
                if x == 1:
                    result_list.append(0)
                else:
                    result_list.append(counter)
                    counter += 1

            else:   
                if input_list[idx-1] == 1 and x == 0:
                    result_list.append(counter)
                    counter += 1
                else:
                    result_list.append(0)

        total.append(result_list)

    공백순서달력 = pd.DataFrame()
    공백순서달력.index = 날짜집합

    for idx, 정반 in enumerate(배치_달력.columns.tolist()):
        공백순서달력[f"{정반}"] =  total[idx]

    return 공백순서달력

# 착수가능일찾기 함수

In [20]:
def 착수가능일찾기(공기달력, 공백순서달력, 정반, 표준공기):
        
    first_zeros = []
    
    for idx, i in enumerate(공백순서달력[f"{정반}"].tolist()):
        if i != 0:
            first_zeros.append(공백순서달력.index.strftime('%Y-%m-%d').values[idx])

    for idx, 착수가능일 in enumerate(first_zeros):
        
        
        착수가능일인덱스 = list(공기달력.index.strftime('%Y-%m-%d')).index(first_zeros[idx])
        착수가능일의확보가능공기 = 공기달력[f"{정반}"].iloc[착수가능일인덱스]
        
        if 착수가능일의확보가능공기 > 표준공기:
            
            return 착수가능일
        
        else:
            pass

# 달력생성 및 업데이트 연습

## 최초달력생성

In [21]:
배치달력 = create_init_calendar(날짜집합, 최초정반집합)
배치달력.iloc[:2,:] = 1   ## 날짜 계산의 편의를 위해 초반 2일을 배치상태로 설정
배치달력

,A,B,C
2024-02-01,1,1,1
2024-02-02,1,1,1
2024-02-03,0,0,0
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


In [22]:
정반집합 = 배치달력. columns.tolist()
공기달력 = create_공기달력(배치달력, 날짜집합, 정반집합)
공기달력

,A,B,C
2024-02-01,0.0,0.0,0.0
2024-02-02,0.0,0.0,0.0
2024-02-03,26.0,26.0,26.0
2024-02-04,25.0,25.0,25.0
2024-02-05,24.0,24.0,24.0
2024-02-06,23.0,23.0,23.0
2024-02-07,22.0,22.0,22.0
2024-02-08,21.0,21.0,21.0
2024-02-09,20.0,20.0,20.0
2024-02-10,19.0,19.0,19.0


In [23]:
공백순서달력 = create_공백순서달력(배치달력, 정반집합, 날짜집합)
공백순서달력

,A,B,C
2024-02-01,0,0,0
2024-02-02,0,0,0
2024-02-03,1,1,1
2024-02-04,0,0,0
2024-02-05,0,0,0
2024-02-06,0,0,0
2024-02-07,0,0,0
2024-02-08,0,0,0
2024-02-09,0,0,0
2024-02-10,0,0,0


## 블록배치

In [26]:
착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, "A", 13)
착수가능일

'2024-02-03'

## 달력 업데이트

In [27]:
새정반집합 = ["A","B","C","A_추가"]
배치달력 = update_배치달력(배치달력, "A", "2024-02-10", 2, 새정반집합)
배치달력

,A,B,C,A_추가
2024-02-01,1,1,1,0
2024-02-02,1,1,1,0
2024-02-03,0,0,0,0
2024-02-04,0,0,0,0
2024-02-05,0,0,0,0
2024-02-06,0,0,0,0
2024-02-07,0,0,0,0
2024-02-08,0,0,0,0
2024-02-09,0,0,0,0
2024-02-10,1,0,0,0


In [28]:
공기달력 = create_공기달력(배치달력, 날짜집합, 새정반집합)
공기달력

,A,B,C,A_추가
2024-02-01,0.0,0.0,0.0,28
2024-02-02,0.0,0.0,0.0,27
2024-02-03,7.0,26.0,26.0,26
2024-02-04,6.0,25.0,25.0,25
2024-02-05,5.0,24.0,24.0,24
2024-02-06,4.0,23.0,23.0,23
2024-02-07,3.0,22.0,22.0,22
2024-02-08,2.0,21.0,21.0,21
2024-02-09,1.0,20.0,20.0,20
2024-02-10,0.0,19.0,19.0,19


In [29]:
공백순서달력 = create_공백순서달력(배치달력, 새정반집합, 날짜집합)
공백순서달력

,A,B,C,A_추가
2024-02-01,0,0,0,1
2024-02-02,0,0,0,0
2024-02-03,1,1,1,0
2024-02-04,0,0,0,0
2024-02-05,0,0,0,0
2024-02-06,0,0,0,0
2024-02-07,0,0,0,0
2024-02-08,0,0,0,0
2024-02-09,0,0,0,0
2024-02-10,0,0,0,0


# 배치시나리오
 - 블록접수
 - 정반 매칭, 배치달력 체크 (우선순위가 고려되는 단계 - 매칭정반, 공기확보 가능한 first_zero)
 - 정반 및 블록 사이즈 체크 - 정반쪼개기
 - 정반새리스트업 - 정반별 배치달력, 공기달력생성
 - 기준시점 생산량 계산 (최적화 목적함수)

## 원데이터 불러오기

In [25]:
블록원데이터 = pd.read_excel("./data/data1.xlsx", sheet_name="블록데이터")
블록원데이터.head()

,블록명,중량,가로,세로,표준공기,납기,정반배치
0,S1,50,10,10,2,2024-02-15,0
1,S2,60,10,12,3,2024-02-15,0
2,S3,30,5,6,1,2024-02-15,0
3,S4,20,5,6,2,2024-02-15,0


In [26]:
정반원데이터 = pd.read_excel("./data/data1.xlsx", sheet_name="정반데이터")
정반원데이터.head()

,정반명,가능중량,가로,세로,occupied
0,A,100,10,20,0
1,B,100,10,15,0
2,C,30,5,10,0


In [27]:
## Initial Settings
start_date = datetime(2024, 2, 1)
end_date = datetime(2024, 2, 28)
날짜집합  = pd.date_range(start=start_date, end=end_date, freq='D')
정반데이터 = 정반데이터전처리(정반원데이터)
최초정반집합 = 정반데이터["정반명"].unique().tolist()
블록데이터 = 블록데이터전처리(블록원데이터)

배치달력 = create_init_calendar(날짜집합, 최초정반집합)
배치달력.iloc[:2,:] = 1
정반집합 = 배치달력. columns.tolist()
정반쪼개는면적_Thresh = 0.3

공기달력 = create_공기달력(배치달력, 날짜집합, 정반집합)
공백순서달력 = create_공백순서달력(배치달력, 정반집합, 날짜집합)

# 결과모음리스트
배정된블록 = []
배정된정반 = []
착수일 = []

for blk in tqdm(블록데이터["블록명"]):

    target_block = blk
    blk_index = 블록데이터[블록데이터["블록명"]==blk].index.values[0]
    target_weight = 블록데이터[블록데이터["블록명"]==target_block]["중량"].values[0]
    target_size = 블록데이터[블록데이터["블록명"]==target_block]["사이즈"].values[0]
    target_표준공기 = 블록데이터[블록데이터["블록명"]==target_block]["표준공기"].values[0]
    print(f"**타겟블록정보 -  idx:{blk_index}, 블록명:{target_block}, 무게:{target_weight}, 사이즈:{target_size}")
    
    가능정반리스트 = 정반데이터[정반데이터["occupied"]==0]["정반명"].tolist()
    print(f"**가능정반리스트 : {가능정반리스트}")
         
    for 가능정반 in 정반데이터[정반데이터["occupied"]==0]["정반명"]:
        
        가능정반인덱스 = 정반데이터[정반데이터["정반명"]==가능정반].index.values[0]
        weight_capa = 정반데이터[정반데이터["정반명"]==가능정반]["가능중량"].values[0]
        size_capa = 정반데이터[정반데이터["정반명"]==가능정반]["사이즈"].values[0]
        print(f"**검토정반 - idx:{가능정반인덱스}, 정반명:{가능정반}, 가능중량:{weight_capa}, 사이즈:{size_capa}")
        
        if weight_capa > target_weight and size_capa > target_size:
            print(f"**{target_block}를 {가능정반}에 배치가능")
            
            # 날짜 및 공기 매칭 체크
            '''
            - 정반의 first zero 날짜가 최소요구착수일 보다 앞서 있고
            - first zero 날짜의 연속확보 가능 공기가, 표준공기보다 길고..
            - 블록이 깔리면 달력을 전부 업데이트 하고... 
            '''
            착수가능일 = 착수가능일찾기(공기달력, 공백순서달력, 가능정반, target_표준공기)
            print(f"**착수가능일: {착수가능일}")
            
            ## 날짜까지 이상없으면 잔여면적 검토후 정반 쪼개기

            잔여면적비율 = (size_capa - target_size) / size_capa
            
            if 잔여면적비율 >= 정반쪼개는면적_Thresh:
                새정반이름 = 가능정반+"_추가"
                새정반면적 = size_capa * 잔여면적비율
                새오큐파이드 = 0
                기존정반새면적 = size_capa - 새정반면적
                
                정반데이터.loc[가능정반인덱스,"사이즈"] = 기존정반새면적
                정반데이터.loc[가능정반인덱스, "occupied"] = 1
                
                블록데이터.loc[blk_index, "정반배치"] = 1
                
                정반데이터.loc[len(정반데이터)] = {"정반명":새정반이름, "가능중량": weight_capa, "사이즈":새정반면적, "occupied":새오큐파이드}
                
                정반데이터["중량순서"] = 정반데이터["가능중량"].rank(ascending=False)
                정반데이터["크기순서"] = 정반데이터["사이즈"].rank(ascending=False)
                정반데이터["우선순위"] = 정반데이터[["중량순서", "크기순서"]].apply(정반우선순위구하기, axis=1)
                정반데이터 = 정반데이터.sort_values(by=["우선순위"])
                
                print(f"**잔여면적비율 {np.round(잔여면적비율,1)*100}%로 30% 이상 정반 쪼개기 - 새이름: {새정반이름} / 새면적: {새정반면적}") 
                print(f"**기존정반새면적: {기존정반새면적}")
            
                ## 새정반 추가사항 달력에 반영
                정반집합.append(새정반이름)    
                새정반집합 = 정반집합
                print(f"**새정반집합: {새정반집합}")
                print("")

                배치달력 =  update_배치달력(배치달력, 가능정반, 착수가능일, target_표준공기, 새정반집합) 
                배치달력.iloc[:2,:] = 1
                공기달력 = create_공기달력(배치달력, 날짜집합, 새정반집합)
                공백순서달력 = create_공백순서달력(배치달력, 새정반집합, 날짜집합)
                
                배정결과 = {"블록명": target_block, "정반명": 가능정반, "착수일": 착수가능일}    
                print(f"최종배정결과 : {배정결과}")
                
                배정된블록.append(target_block)
                배정된정반.append(가능정반)
                착수일.append(착수가능일)
            
                break      
            else:
                pass
        else:
            pass
        
        print("-"*50)
    print("="*70)

100%|██████████████████████████████| 4/4 [00:00<00:00, 44.45it/s]

**타겟블록정보 -  idx:1, 블록명:S2, 무게:60, 사이즈:120
**가능정반리스트 : ['A', 'B', 'C']
**검토정반 - idx:0, 정반명:A, 가능중량:100, 사이즈:200
**S2를 A에 배치가능
**착수가능일: 2024-02-03
**잔여면적비율 40.0%로 30% 이상 정반 쪼개기 - 새이름: A_추가 / 새면적: 80.0
**기존정반새면적: 120.0
**새정반집합: ['A', 'B', 'C', 'A_추가']

최종배정결과 : {'블록명': 'S2', '정반명': 'A', '착수일': '2024-02-03'}
**타겟블록정보 -  idx:0, 블록명:S1, 무게:50, 사이즈:100
**가능정반리스트 : ['B', 'A_추가', 'C']
**검토정반 - idx:1, 정반명:B, 가능중량:100, 사이즈:150.0
**S1를 B에 배치가능
**착수가능일: 2024-02-03
**잔여면적비율 30.0%로 30% 이상 정반 쪼개기 - 새이름: B_추가 / 새면적: 50.0
**기존정반새면적: 100.0
**새정반집합: ['A', 'B', 'C', 'A_추가', 'B_추가']

최종배정결과 : {'블록명': 'S1', '정반명': 'B', '착수일': '2024-02-03'}
**타겟블록정보 -  idx:3, 블록명:S4, 무게:20, 사이즈:30
**가능정반리스트 : ['A_추가', 'B_추가', 'C']
**검토정반 - idx:3, 정반명:A_추가, 가능중량:100, 사이즈:80.0
**S4를 A_추가에 배치가능
**착수가능일: 2024-02-03
**잔여면적비율 60.0%로 30% 이상 정반 쪼개기 - 새이름: A_추가_추가 / 새면적: 50.0
**기존정반새면적: 30.0
**새정반집합: ['A', 'B', 'C', 'A_추가', 'B_추가', 'A_추가_추가']

최종배정결과 : {'블록명': 'S4', '정반명': 'A_추가', '착수일': '2024-02-03'}
**타겟블록정보 -  idx:2, 블록명:S3, 무게:30,

In [28]:
블록정반배정결과 = pd.DataFrame({"블록명":배정된블록, "정반명":배정된정반, "착수일":착수일})
블록정반배정결과  # 이걸로 시점별 목적함수(생산량)을 계산할 수 있다.

,블록명,정반명,착수일
0,S2,A,2024-02-03
1,S1,B,2024-02-03
2,S4,A_추가,2024-02-03
3,S3,B_추가,2024-02-03


In [41]:
정반데이터  #분할후 신규생선된 정반이 포함되어 있음

,정반명,가능중량,가로,세로,occupied,사이즈,중량순서,크기순서,우선순위
2,C,100,10.0,20.0,1,80.0,6.5,4.0,2.0
1,B,100,10.0,20.0,1,80.0,6.5,4.0,2.0
0,A,100,10.0,20.0,1,80.0,6.5,4.0,2.0
15,C_추가_추가,100,NaN,NaN,1,70.0,6.5,8.0,2.9
13,A_추가_추가,100,NaN,NaN,1,70.0,6.5,8.0,2.9
14,B_추가_추가,100,NaN,NaN,1,70.0,6.5,8.0,2.9
7,H,30,10.0,20.0,0,200.0,19.5,1.5,3.6
8,I,30,10.0,20.0,0,200.0,19.5,1.5,3.6
3,D,50,10.0,10.0,1,70.0,14.5,8.0,4.3
9,J,50,10.0,10.0,1,70.0,14.5,8.0,4.3


In [39]:
블록데이터

,블록명,중량,가로,세로,표준공기,납기,정반배치,사이즈,최소요구착수일,날짜순서,공기순서,크기순서,우선순위
6,S7,60,10,12,3,2024-02-05,1,120,2024-02-02,1.0,3.0,3.0,1.4
8,S9,60,10,12,3,2024-02-10,1,120,2024-02-07,2.0,3.0,3.0,1.7
9,S10,60,10,12,3,2024-02-12,1,120,2024-02-09,8.0,3.0,3.0,3.1
7,S8,60,10,12,3,2024-02-13,0,120,2024-02-10,10.5,3.0,3.0,3.6
10,S11,60,10,12,3,2024-02-15,0,120,2024-02-12,13.0,3.0,3.0,4.2
0,S1,50,10,10,2,2024-02-10,1,100,2024-02-08,4.5,10.5,8.5,4.8
2,S3,50,10,10,2,2024-02-10,1,100,2024-02-08,4.5,10.5,8.5,4.8
1,S2,50,10,10,2,2024-02-10,1,100,2024-02-08,4.5,10.5,8.5,4.8
16,S17,20,5,6,2,2024-02-10,1,30,2024-02-08,4.5,10.5,16.0,6.5
4,S5,50,10,10,2,2024-02-15,0,100,2024-02-13,16.0,10.5,8.5,7.5


In [40]:
배치달력

,A,B,C,H,I,D,J,E,F,G,...,C_추가,A_추가_추가,B_추가_추가,C_추가_추가,A_추가_추가_추가,B_추가_추가_추가,C_추가_추가_추가,J_추가,D_추가,F_추가
2024-02-01,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,0,0,0,0,0,0
2024-02-02,1,1,1,1,1,1,1,1,1,1,...,1,1,1,0,0,0,0,0,0,0
2024-02-03,1,1,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-04,1,1,1,0,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-05,1,1,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-06,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-07,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-08,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-09,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2024-02-10,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
